In [38]:
from langchain.llms import GooglePalm
from langchain.chat_models import ChatGooglePalm
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback

import os
import json
import pandas as pd
import traceback

from dotenv import load_dotenv
import PyPDF2

In [39]:
load_dotenv()

True

In [40]:
google_api_key=os.getenv("GOOGLE_API_KEY")

In [41]:
llm=ChatGooglePalm(model_name="models/chat-bison-001",google_api_key=google_api_key,temperature=0.7)

In [42]:
llm

ChatGooglePalm(client=<module 'google.generativeai' from 'd:\\MCQ Generator Project\\venv\\lib\\site-packages\\google\\generativeai\\__init__.py'>, google_api_key=SecretStr('**********'), temperature=0.7)

In [43]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [44]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",

    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",

    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",

    },
}

In [45]:
response_json=json.dumps(RESPONSE_JSON)

In [46]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [47]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [48]:
PATH="./../data.txt"

with open(PATH,'r') as f:
    TEXT=f.read()

In [49]:
NUMBER=5
SUBJECT='Biology'
TONE='simple'


In [50]:
quiz_result = quiz_chain.run({
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    })



> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research question

In [51]:
quiz_result

"### Biology MCQ Quiz\n\n1. **What is biology?**\n\nA. The scientific study of life.\nB. The study of the Earth's physical environment.\nC. The study of the human body.\nD. The study of plants.\n\n2. **What are some unifying themes in biology?**\n\nA. All organisms are made up of cells.\nB. All organisms process hereditary information encoded in genes.\nC. All organisms are able to move, grow, and reproduce.\nD. All of the above.\n\n3. **What is evolution?**\n\nA. The process by which species change over time.\nB. The process by which organisms adapt to their environment.\nC. The process by which new species are formed.\nD. All of the above.\n\n4. **What is energy processing?**\n\nA. The process by which organisms convert energy from one form to another.\nB. The process by which organisms use energy to move, grow, and reproduce.\nC. The process by which organisms maintain their internal environment.\nD. All of the above.\n\n5. **What is homeostasis?**\n\nA. The ability of an organism t

In [52]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:

"""

In [53]:
quiz_evaluation_prompt=PromptTemplate(input_variables=['subject','quiz'],template=TEMPLATE2)

In [54]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [55]:
review_result=review_chain.run({
    "subject":SUBJECT,
    "quiz":quiz_result
})



> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for Biology students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
### Biology MCQ Quiz

1. **What is biology?**

A. The scientific study of life.
B. The study of the Earth's physical environment.
C. The study of the human body.
D. The study of plants.

2. **What are some unifying themes in biology?**

A. All organisms are made up of cells.
B. All organisms process hereditary information encoded in genes.
C. All organisms are able to move, grow, and reproduce.
D. All of the above.

3. **What is evolution?**

A. The process by which speci

In [56]:
review_result

'The quiz is well-written and clear, with no grammatical or spelling errors. The questions are all relevant to the topic of biology, and the answers are accurate. The quiz is also at a suitable level of difficulty for high school students.\n\nHowever, there are a few areas where the quiz could be improved. First, the quiz could be more engaging by including more visuals, such as diagrams or images. Second, the quiz could be more interactive by including drag-and-drop or fill-in-the-blank questions. Finally, the quiz could be more personalized by including questions that are tailored to the student\'s individual interests.\n\nOverall, the quiz is a valuable tool for assessing students\' knowledge of biology. With a few minor improvements, it could be an even more effective learning tool.\n\nHere are some specific suggestions for improvement:\n\n1. **Include more visuals.** Diagrams, images, and other visuals can help students to understand complex concepts more easily. For example, the 

In [57]:
generate_evaluation_chain=SequentialChain(chains=[quiz_chain,review_chain],
                input_variables=["text","number","subject","tone","response_json"],
                output_variables=['quiz','review'],
                verbose=True)

In [58]:
PATH="./../data.txt"

In [59]:
with open(PATH,'r') as file :
    TEXT=file.read()

In [60]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [61]:
# TEXT
NUMBER=5
SUBJECT="Biology"
TONE='simple'
# RESPONSE_JOSN=RESPONSE_JOSN

In [62]:
# json.dumps(RESPONSE_JSON)

In [63]:

with get_openai_callback() as cb:
    response=generate_evaluation_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )






> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [68]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [65]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [66]:
quiz=response.get('quiz')

In [67]:
quiz

'Sure, here are 5 multiple choice questions for Biology students in simple tone:\n\n1. Biology is the scientific study of:\n    * A. the universe\n    * B. life\n    * C. the human body\n    * D. the Earth\n\n2. All organisms are made up of:\n    * A. cells\n    * B. atoms\n    * C. molecules\n    * D. tissues\n\n3. Energy processing is important to life because it allows organisms to:\n    * A. move\n    * B. grow\n    * C. reproduce\n    * D. all of the above\n\n4. Biologists study life at multiple levels of organization, including:\n    * A. the molecular biology of a cell\n    * B. the anatomy and physiology of plants and animals\n    * C. the evolution of populations\n    * D. all of the above\n\n5. Life on Earth, which emerged more than 3.7 billion years ago, is immensely diverse. Biologists have sought to study and classify the various forms of life, including:\n    * A. prokaryotic organisms such as archaea and bacteria\n    * B. eukaryotic organisms such as protists, fungi, pl

In [69]:
# Parse the questions, choices, and answers
questions = []
choices = []
answers = []




In [70]:
lines = quiz.split('\n')
i = 0
while i < len(lines):
    line = lines[i].strip()
    if line and line[0].isdigit():
        question = line.split(':')[0].strip()
        choice = ""
        i += 1
        while i < len(lines) and lines[i].strip().startswith("*"):
            choice += lines[i].strip() + " | "
            i += 1
        questions.append(question)
        choices.append(choice.strip(" | "))
    else:
        i += 1

In [80]:
questions=questions[:5]

In [81]:
choices=choices[:5]

In [78]:
# Parse the answers
answers_section = lines.index("The answers are:") + 1
for line in lines[answers_section:]:
    if line.strip():
        q_num, answer = line.strip().split(". ")
        answers.append(answer)

In [79]:
answers

['B', 'A', 'D', 'D', 'C']

In [91]:
([data])

[{'MCQ': ['1. Biology is the scientific study of',
   '2. All organisms are made up of',
   '3. Energy processing is important to life because it allows organisms to',
   '4. Biologists study life at multiple levels of organization, including',
   '5. Life on Earth, which emerged more than 3.7 billion years ago, is immensely diverse. Biologists have sought to study and classify the various forms of life, including'],
  'Choices': ['* A. the universe | * B. life | * C. the human body | * D. the Earth',
   '* A. cells | * B. atoms | * C. molecules | * D. tissues',
   '* A. move | * B. grow | * C. reproduce | * D. all of the above',
   '* A. the molecular biology of a cell | * B. the anatomy and physiology of plants and animals | * C. the evolution of populations | * D. all of the above',
   '* A. prokaryotic organisms such as archaea and bacteria | * B. eukaryotic organisms such as protists, fungi, plants, and animals | * C. both prokaryotic and eukaryotic organisms | * D. none of the ab

In [85]:
data={
    "MCQ":questions,
    "Choices":choices,
    "Answers":answers
}

df = pd.DataFrame(data)

In [86]:
df.head()

,MCQ,Choices,Answers
0,1. Biology is the scientific study of,* A. the universe | * B. life | * C. the human...,B
1,2. All organisms are made up of,* A. cells | * B. atoms | * C. molecules | * D...,A
2,3. Energy processing is important to life beca...,* A. move | * B. grow | * C. reproduce | * D. ...,D
3,4. Biologists study life at multiple levels of...,* A. the molecular biology of a cell | * B. th...,D
4,"5. Life on Earth, which emerged more than 3.7 ...",* A. prokaryotic organisms such as archaea and...,C


In [87]:
df.to_csv('biology_question.csv',index=False)

In [88]:
from datetime import datetime

datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'06_26_2024_16_47_46'